In [1]:
%run helper/setup_notebook.ipynb import display_table

Successfully connected to leetcode50 database.


In [2]:
tables = ['Signups', 'Confirmations']

for table in tables:
    display_table(table)

+---------+---------------------+
| user_id |      time_stamp     |
+---------+---------------------+
|    3    | 2020-03-21 10:16:13 |
|    7    | 2020-01-04 13:57:59 |
|    2    | 2020-07-29 23:09:44 |
|    6    | 2020-12-09 10:39:37 |
+---------+---------------------+
+---------+---------------------+-----------+
| user_id |      time_stamp     |   action  |
+---------+---------------------+-----------+
|    3    | 2021-01-06 03:30:46 |  timeout  |
|    3    | 2021-07-14 14:00:00 |  timeout  |
|    7    | 2021-06-12 11:57:29 | confirmed |
|    7    | 2021-06-13 12:58:28 | confirmed |
|    7    | 2021-06-14 13:59:27 | confirmed |
|    2    | 2021-01-22 00:00:00 | confirmed |
|    2    | 2021-02-28 23:59:59 |  timeout  |
+---------+---------------------+-----------+


### The confirmation rate of a user is the number of 'confirmed' messages divided by the total number of requested confirmation messages. The confirmation rate of a user that did not request any confirmation messages is 0. Round the confirmation rate to two decimal places. Write an SQL query to find the confirmation rate of each user.

```
+---------+-------------------+
| user_id | confirmation_rate |
+---------+-------------------+
| 6       | 0.00              |
| 3       | 0.00              |
| 7       | 1.00              |
| 2       | 0.50              |
+---------+-------------------+
Explanation: 
User 6 did not request any confirmation messages. The confirmation rate is 0.
User 3 made 2 requests and both timed out. The confirmation rate is 0.
User 7 made 3 requests and all were confirmed. The confirmation rate is 1.
User 2 made 2 requests where one was confirmed and the other timed out. The confirmation rate is 1 / 2 = 0.5.
```

In [3]:
%%sql 

SELECT 
    s.user_id,
    c.action,
    COUNT(c.action) AS total_count
FROM Confirmations c 
RIGHT JOIN Signups s ON s.user_id = c.user_id
GROUP BY s.user_id, c.action

user_id,action,total_count
3,timeout,2
7,confirmed,3
2,timeout,1
2,confirmed,1
6,None,0


In [4]:
%%sql 

SELECT 
    s.user_id,
    COUNT(c.action) AS total_count
FROM Confirmations c 
RIGHT JOIN Signups s ON s.user_id = c.user_id
GROUP BY s.user_id

user_id,total_count
3,2
7,3
2,2
6,0


In [5]:
%%sql 
-- the issue with the query is that user2 still needs to be aggreaged to reflect the correct total_count
SELECT 
    s.user_id, 
    COUNT(c.action) AS total_count,
    CASE WHEN c.action = 'confirmed' THEN COUNT(c.action) ELSE 0 END AS confirmed_count,
    CASE WHEN c.action = 'timeout' THEN COUNT(c.action) ELSE 0 END AS timeout_count
FROM Confirmations c 
RIGHT JOIN Signups s ON s.user_id = c.user_id
GROUP BY s.user_id, c.action

user_id,total_count,confirmed_count,timeout_count
3,2,0,2
7,3,3,0
2,1,0,1
2,1,1,0
6,0,0,0


In [6]:
%%sql 

SELECT 
    user_id, 
    SUM(total_count),
    SUM(confirmed_count),
    SUM(timeout_count)
FROM 
    (
    SELECT 
        s.user_id, 
        COUNT(c.action) AS total_count,
        CASE WHEN c.action = 'confirmed' THEN COUNT(c.action) ELSE 0 END AS confirmed_count,
        CASE WHEN c.action = 'timeout' THEN COUNT(c.action) ELSE 0 END AS timeout_count
    FROM Confirmations c 
    RIGHT JOIN Signups s ON s.user_id = c.user_id
    GROUP BY s.user_id, c.action
    ) inner_query 
GROUP BY user_id

user_id,SUM(total_count),SUM(confirmed_count),SUM(timeout_count)
3,2,0,2
7,3,3,0
2,2,1,1
6,0,0,0


In [7]:
%%sql 
-- calculate the confirmation_rate 

SELECT 
    user_id, 
    (SUM(confirmed_count)/ SUM(total_count)) AS confirmation_rate
FROM 
    (
    SELECT 
        s.user_id, 
        COUNT(c.action) AS total_count,
        CASE WHEN c.action = 'confirmed' THEN COUNT(c.action) ELSE 0 END AS confirmed_count,
        CASE WHEN c.action = 'timeout' THEN COUNT(c.action) ELSE 0 END AS timeout_count
    FROM Confirmations c 
    RIGHT JOIN Signups s ON s.user_id = c.user_id
    GROUP BY s.user_id, c.action
    ) inner_query 
GROUP BY user_id

user_id,confirmation_rate
3,0.0000
7,1.0000
2,0.5000
6,None


In [8]:
%%sql 
-- replace null values with 0 

SELECT 
    user_id, 
    COALESCE((SUM(confirmed_count)/ SUM(total_count)), 0) AS confirmation_rate
FROM 
    (
    SELECT 
        s.user_id, 
        COUNT(c.action) AS total_count,
        CASE WHEN c.action = 'confirmed' THEN COUNT(c.action) ELSE 0 END AS confirmed_count,
        CASE WHEN c.action = 'timeout' THEN COUNT(c.action) ELSE 0 END AS timeout_count
    FROM Confirmations c 
    RIGHT JOIN Signups s ON s.user_id = c.user_id
    GROUP BY s.user_id, c.action
    ) inner_query 
GROUP BY user_id

user_id,confirmation_rate
3,0.0000
7,1.0000
2,0.5000
6,0.0000


In [9]:
%%sql 
-- round up to 2 decimal points

SELECT 
    user_id, 
    ROUND(COALESCE((SUM(confirmed_count)/ SUM(total_count)), 0), 2) AS confirmation_rate
FROM 
    (
    SELECT 
        s.user_id, 
        COUNT(c.action) AS total_count,
        CASE WHEN c.action = 'confirmed' THEN COUNT(c.action) ELSE 0 END AS confirmed_count,
        CASE WHEN c.action = 'timeout' THEN COUNT(c.action) ELSE 0 END AS timeout_count
    FROM Confirmations c 
    RIGHT JOIN Signups s ON s.user_id = c.user_id
    GROUP BY s.user_id, c.action
    ) inner_query 
GROUP BY user_id

user_id,confirmation_rate
3,0.00
7,1.00
2,0.50
6,0.00


# Refactored Query

In [10]:
%%sql 

SELECT 
    s.user_id, 
    ROUND(COALESCE((SUM(c.action = 'confirmed')/ SUM(c.action IN ('confirmed', 'timeout'))), 0), 2) AS confirmation_rate
FROM Signups s 
LEFT JOIN Confirmations c ON s.user_id = c.user_id 
GROUP BY s.user_id;

user_id,confirmation_rate
3,0.00
7,1.00
2,0.50
6,0.00


## Improved Query #1

In [11]:
%%sql 

SELECT 
    s.user_id, 
    AVG(IF(c.action="confirmed",1,0)) AS confirmation_rate
FROM Confirmations c 
RIGHT JOIN Signups s ON s.user_id = c.user_id
GROUP BY s.user_id;

user_id,confirmation_rate
3,0.0000
7,1.0000
2,0.5000
6,0.0000


In [12]:
%%sql 

SELECT 
    s.user_id, 
    ROUND(AVG(IF(c.action="confirmed",1,0)), 2) AS confirmation_rate
FROM Confirmations c 
RIGHT JOIN Signups s ON s.user_id = c.user_id
GROUP BY s.user_id;

user_id,confirmation_rate
3,0.00
7,1.00
2,0.50
6,0.00


## Improved Query #2

In [13]:
%%sql

SELECT 
  s.user_id,
  ROUND(AVG(CASE WHEN action = 'confirmed' THEN 1.00 ELSE 0.00 END),2) AS confirmation_rate
FROM Confirmations c 
RIGHT JOIN Signups s ON s.user_id = c.user_id
GROUP BY s.user_id;

user_id,confirmation_rate
3,0.00
7,1.00
2,0.50
6,0.00


# Using Pandas

In [14]:
import pandas as pd 

In [15]:
confirmation_query = %sql SELECT * FROM Confirmations # type: ignore
signup_query = %sql SELECT * FROM Signups # type: ignore

confirmation_df = confirmation_query.DataFrame()
signup_df = signup_query.DataFrame()

display(confirmation_df, signup_df)

,user_id,time_stamp,action
0,3,2021-01-06 03:30:46,timeout
1,3,2021-07-14 14:00:00,timeout
2,7,2021-06-12 11:57:29,confirmed
3,7,2021-06-13 12:58:28,confirmed
4,7,2021-06-14 13:59:27,confirmed
5,2,2021-01-22 00:00:00,confirmed
6,2,2021-02-28 23:59:59,timeout


,user_id,time_stamp
0,3,2020-03-21 10:16:13
1,7,2020-01-04 13:57:59
2,2,2020-07-29 23:09:44
3,6,2020-12-09 10:39:37


In [16]:
confirmation_df.merge(signup_df, on='user_id', how='right')

,user_id,time_stamp_x,action,time_stamp_y
0,3,2021-01-06 03:30:46,timeout,2020-03-21 10:16:13
1,3,2021-07-14 14:00:00,timeout,2020-03-21 10:16:13
2,7,2021-06-12 11:57:29,confirmed,2020-01-04 13:57:59
3,7,2021-06-13 12:58:28,confirmed,2020-01-04 13:57:59
4,7,2021-06-14 13:59:27,confirmed,2020-01-04 13:57:59
5,2,2021-01-22 00:00:00,confirmed,2020-07-29 23:09:44
6,2,2021-02-28 23:59:59,timeout,2020-07-29 23:09:44
7,6,NaT,NaN,2020-12-09 10:39:37


In [17]:
merged_df = confirmation_df.merge(signup_df, on='user_id', how='right')
merged_df.groupby('user_id').agg({'user_id': 'count'})

,user_id
user_id,
2,2
3,2
6,1
7,3


In [18]:
merged_df.groupby('user_id')[['action']] \
    .apply(lambda x: (x == 'confirmed').mean())

,action
user_id,
2,0.5
3,0.0
6,0.0
7,1.0


In [19]:
confirmation_rate_df = merged_df.groupby('user_id')[['action']] \
    .apply(lambda x: (x == 'confirmed').mean()) \
        .reset_index()
confirmation_rate_df

,user_id,action
0,2,0.5
1,3,0.0
2,6,0.0
3,7,1.0


In [20]:
confirmation_rate_df = confirmation_rate_df.rename(columns={'action':'confrimation_rate'}).round(2)
confirmation_rate_df

,user_id,confrimation_rate
0,2,0.5
1,3,0.0
2,6,0.0
3,7,1.0
